In [63]:
import os
import gensim
from gensim import models
from gensim.models import Word2Vec, KeyedVectors
from gensim.models import Phrases

In [18]:
import psycopg2

con = psycopg2.connect(
  database="db_dsa", 
  user="udsa", 
  password="P@ssword2021", 
  host="92.242.58.173", 
  port="1984"
)

print("Database opened successfully")

Database opened successfully


In [25]:
cur = con.cursor()
cur.execute("SELECT * from PROJECTS")
rows = cur.fetchall()
column_names=["ID","NUMBER","VACANCIES","NAME","TYPE","TYPEDESC","TYPEID","STATUSID", "STATUSDESC","DIRECTIONHEAD","HEAD","DATACREATED","FACULTYID"]
df = pd.DataFrame(rows, columns=column_names)
df.head()

,ID,NUMBER,VACANCIES,NAME,TYPE,TYPEDESC,TYPEID,STATUSID,STATUSDESC,DIRECTIONHEAD,HEAD,DATACREATED,FACULTYID
0,205,NaN,2,Численное и аналитическое исследование эффекта...,nir,НИР,1,1,Готов к работе,Данилов Владимир Григорьевич,Данилов Владимир Григорьевич,08.05.2020 21:25:02,None
1,208,NaN,3,"Исследование механизмов образования, способов ...",nir,НИР,1,1,Готов к работе,,Грачев Николай,11.05.2020 14:07:40,None
2,215,NaN,1,Разработка моста переменного тока,soft-hard,Прогр-аппарат.,3,1,Готов к работе,,Балакин Станислав,13.05.2020 16:41:39,None
3,216,NaN,3,Разработка нательных антенн и излучателей для ...,soft-hard,Прогр-аппарат.,3,1,Готов к работе,Елизаров Андрей Альбертович,Скуридин Андрей,13.05.2020 17:00:19,None
4,218,NaN,0,Разработка глоссария физических терминов для м...,soft,Прогр.,2,1,Готов к работе,,Гузенкова Александра Сергеевна,13.05.2020 19:38:43,None


In [84]:
#подготовили данные
df1 = df[['VACANCIES','NAME','TYPE', 'TYPEDESC', 'TYPEID']]
df2 = df1.apply(lambda x: ';'.join(x.astype(str)), axis=1)
df_clean = pd.DataFrame({'clean': df2})
sent = [row.split(';') for row in df_clean['clean']]
sent[0][1]

'Численное и аналитическое исследование эффекта Саффмана-Тейлора'

In [59]:
model = Word2Vec(sent, min_count=1, workers=3, window=3, sg=1)

In [61]:
model.wv['Численное и аналитическое исследование эффекта Саффмана-Тейлора']

array([ 9.6586719e-03,  6.5849605e-03, -9.2384471e-03, -7.1667768e-03,
       -8.3938958e-03,  4.2208377e-03,  9.1112992e-03, -5.2192984e-03,
       -8.8882614e-03, -1.3697541e-03, -2.7786859e-03,  1.2769294e-03,
        5.3007063e-03,  5.3801591e-04,  4.7876956e-03,  9.1980742e-03,
       -7.8712506e-03,  5.7810997e-03,  5.9371360e-04, -8.1258845e-03,
       -5.3858776e-03,  4.2189239e-03,  7.9421699e-03, -6.3709021e-03,
       -4.3968405e-03, -2.6843355e-03, -3.4848482e-03, -9.5763905e-03,
        1.5798631e-03, -7.6085483e-03, -6.2058717e-03,  4.3801148e-03,
       -5.6908480e-03, -3.0399514e-03,  6.1912421e-04,  9.3585057e-03,
       -5.4512424e-03,  8.6868758e-04, -6.9878018e-03,  6.5162652e-03,
        9.9316034e-03,  9.8345680e-03, -4.1456572e-03,  9.5488019e-03,
        7.1329605e-03, -2.2072163e-04, -3.6325033e-03,  1.5694521e-03,
        3.7358152e-03, -7.2864215e-03,  3.4509320e-03,  2.9566977e-03,
        8.4105245e-04, -2.8386281e-03,  2.3977007e-03,  5.5984347e-03,
      

In [69]:
model.wv.most_similar('Численное и аналитическое исследование эффекта Саффмана-Тейлора', topn=10)

[('Разработка онлайн конструктора курсов (дисциплин)', 0.2933916449546814),
 ('Self-learning Monte-Carlo algorithms', 0.2611086368560791),
 ('Smart медицина', 0.23156313598155975),
 ('Статистические свойства случайных блужданий с объемным и поверхностным подкреплением',
  0.22982531785964966),
 ('Indoor - позиционирование', 0.21390990912914276),
 ('Моделирование электрических режимов и конструкции канала регулирования соотношения компонентов топлива ракетного двигателя',
  0.20061825215816498),
 ('Доверенная клавиатура', 0.19228802621364594),
 ('Разработка автоматизированного классификатора коротких текстов',
  0.19188471138477325),
 ('1', 0.18962623178958893),
 ('Автоматизация процессов распределения продукции на основе спроса',
  0.18877863883972168)]